# Imports

In [15]:
import ipyleaflet
import geopandas as gpd
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import base64
from shapely import Point
import ipywidgets
import bqplot.pyplot as plt
import xarray as xr


# Open File in GeoPandas

In [16]:
# Open w/ xarray
nc_file = 'download_Xsmall.nc'
ds = xr.open_dataset(nc_file)
ds

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801, time: 3)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2022-12-29T12:00:00 ... 2022-12-31T12:00:00
Data variables:
    sd         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-22 20:53:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...


# Extract Snow Water Equivalent

In [2]:
sdwe = ds['sd']
sdwe

<xarray.DataArray 'sd' (time: 3, latitude: 1801, longitude: 3600)>
[19450800 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2022-12-29T12:00:00 ... 2022-12-31T12:00:00
Attributes:
    units:          m of water equivalent
    long_name:      Snow depth
    standard_name:  lwe_thickness_of_surface_snow_amount

# Convert to Pandas Dataframe

In [3]:
df = sdwe.to_dataframe().reset_index()
df

,time,latitude,longitude,sd
0,2022-12-29 12:00:00,90.0,0.000000,NaN
1,2022-12-29 12:00:00,90.0,0.100000,NaN
2,2022-12-29 12:00:00,90.0,0.200000,NaN
3,2022-12-29 12:00:00,90.0,0.300000,NaN
4,2022-12-29 12:00:00,90.0,0.400000,NaN
...,...,...,...,...
19450795,2022-12-31 12:00:00,-90.0,359.500000,10.0
19450796,2022-12-31 12:00:00,-90.0,359.600006,10.0
19450797,2022-12-31 12:00:00,-90.0,359.700012,10.0
19450798,2022-12-31 12:00:00,-90.0,359.799988,10.0


# Convert to Geopandas Dataframe 

In [4]:
gdf = gpd.GeoDataFrame(
    df[["sd", "time"]],
    geometry=gpd.points_from_xy(df.longitude,df.latitude)
)

gdf

,sd,time,geometry
0,NaN,2022-12-29 12:00:00,POINT (0.000 90.000)
1,NaN,2022-12-29 12:00:00,POINT (0.100 90.000)
2,NaN,2022-12-29 12:00:00,POINT (0.200 90.000)
3,NaN,2022-12-29 12:00:00,POINT (0.300 90.000)
4,NaN,2022-12-29 12:00:00,POINT (0.400 90.000)
...,...,...,...
19450795,10.0,2022-12-31 12:00:00,POINT (359.500 -90.000)
19450796,10.0,2022-12-31 12:00:00,POINT (359.600 -90.000)
19450797,10.0,2022-12-31 12:00:00,POINT (359.700 -90.000)
19450798,10.0,2022-12-31 12:00:00,POINT (359.800 -90.000)


# Filter out null values

In [5]:
gdf = gdf[~gdf["sd"].isna()]
gdf

,sd,time,geometry
237234,10.0,2022-12-29 12:00:00,POINT (323.400 83.500)
237235,10.0,2022-12-29 12:00:00,POINT (323.500 83.500)
237236,10.0,2022-12-29 12:00:00,POINT (323.600 83.500)
237237,10.0,2022-12-29 12:00:00,POINT (323.700 83.500)
237238,10.0,2022-12-29 12:00:00,POINT (323.800 83.500)
...,...,...,...
19450795,10.0,2022-12-31 12:00:00,POINT (359.500 -90.000)
19450796,10.0,2022-12-31 12:00:00,POINT (359.600 -90.000)
19450797,10.0,2022-12-31 12:00:00,POINT (359.700 -90.000)
19450798,10.0,2022-12-31 12:00:00,POINT (359.800 -90.000)


# Filter out 10's (too many to not be suspicous)

In [6]:
gdf = gdf[gdf["sd"]!=10]
gdf

,sd,time,geometry
244398,8.418507,2022-12-29 12:00:00,POINT (319.800 83.300)
244402,0.569179,2022-12-29 12:00:00,POINT (320.200 83.300)
244403,0.216990,2022-12-29 12:00:00,POINT (320.300 83.300)
244404,0.339066,2022-12-29 12:00:00,POINT (320.400 83.300)
244405,0.429860,2022-12-29 12:00:00,POINT (320.500 83.300)
...,...,...,...
19031572,9.487129,2022-12-31 12:00:00,POINT (197.200 -78.400)
19031573,9.486519,2022-12-31 12:00:00,POINT (197.300 -78.400)
19031574,9.486977,2022-12-31 12:00:00,POINT (197.400 -78.400)
19031575,9.488350,2022-12-31 12:00:00,POINT (197.500 -78.400)


# Print Summary Statistics

In [7]:
gdf_reset_index = gdf.reset_index()

numeric_columns = gdf_reset_index.select_dtypes(include=['float32', 'int64'])
summary_stats = numeric_columns['sd'].describe()
print(summary_stats)
print(gdf_reset_index.columns)

count    4.419897e+06
mean     8.755782e-02
std      6.025451e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.525879e-04
75%      6.363201e-02
max      9.999542e+00
Name: sd, dtype: float64
Index(['index', 'sd', 'time', 'geometry'], dtype='object')


# Define Functions

In [11]:
def create_plot(data):
    fig = plt.figure()
    plt.plot( data['time'], data['sd'] )
    fig.layout.width = "325px"
    fig.layout.height = "250px"
    return fig

def get_closest_points(lat, lon):
    clicked_point = Point(lat, lon)
    point_dist = gdf.distance(clicked_point)
    closest_points = gdf[point_dist==point_dist.min()]
    return closest_points

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        
        #get coordinates of mouseclick
        coords = kwargs.get('coordinates')
        lat = coords[0]
        lon = coords[1]
        
        closest_points = get_closest_points(lat, lon)
        
        fig = create_plot(closest_points)

        # create marker and insert plot into popup
        marker = ipyleaflet.Marker(location=coords, popup=ipyleaflet.Popup(child=fig, width=500, height=300))
        
        m.add_layer(marker)


# Call Map

In [17]:
m = ipyleaflet.Map()

m.on_interaction(handle_click)

m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…